In [7]:
!git clone https://github.com/kowndinya-renduchintala/POSIX.git
%cd POSIX
!pip install -e .
!pip install .
!pip install transformers accelerate

Cloning into 'POSIX'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 31 (delta 9), reused 23 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 675.84 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/POSIX/POSIX
Obtaining file:///content/POSIX/POSIX
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for prompt-sensitivity-index (pyproject.toml) ... done
  Created wheel for prompt-sensitivity-index: filename=prompt_sensitivity_index-0.0.1-0.editable-py3-none-any.whl size=5292 sha256=dd3b1008a6acad8525ef13565ea7c774ab025d6ab463634c27a457bd7508b071
  Stored in directory: /tmp/pip-ephem-wheel-cache-r30_zu2d/wheels/fa/5e/95/6278f1d3c846c1af6fd590228d657b3d01af6ac3c2e0ef3a7e
Succes

In [3]:
from huggingface_hub import login
login()

In [8]:
from prompt_sensitivity_index.models import HFModel
from prompt_sensitivity_index.posix import (
    PosixConfig,
    get_prompt_sensitivity,
    write_trace_to_json
)
import json

In [9]:
#example from their repo - https://github.com/kowndinya-renduchintala/POSIX

# Example intent-aligned prompt sets
intent_aligned_prompts = [
    [
        "Q: What is the capital of France?\nA: ",
        "Q: WHat is te capital city of France?\nA: ",
        "Q: what is teh cpital of france??\nA: "
    ],
    [
        "Q: What is the national animal of India?\nA: ",
        "Q: What's the national animl of India?\nA: ",
        "Q: WHat is teh national animal of India?\nA: "
    ],
    [
        "Q: Tell me the meaning of rendezvous?\nA: ",
        "Q: WHat is the meaning of rendezvous?\nA: ",
        "Q: What does rendezvous mean?\nA: "
    ]
]

model_name = "openai-community/gpt2"
model = HFModel(model_name)
config = PosixConfig(max_new_tokens=10, batched=False)
posix_score, trace = get_prompt_sensitivity(model, intent_aligned_prompts, config, verbose=True)
print(f"Prompt Sensitivity Index: {posix_score:.4f}")
write_trace_to_json(trace, "posix_trace.json")

# Show first few prompts, responses, and logprobs
for i, group in enumerate(trace.prompt_sensitivities):
    print(f"\n--- Prompt Group {i+1} ---")
    for j, prompt in enumerate(trace.prompts[i]):
        print(f"Prompt: {prompt}")
        print(f"Response: {trace.responses[i][j]}")
        print(f"Logprobs: {trace.logprob_matrices[i][j]}")
    print(f"Sensitivity: {trace.prompt_sensitivities[i]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting pad token to <|endoftext|>
Total Number of Intent-Aligned Prompt Sets: 3


  0%|          | 0/3 [00:00<?, ?it/s]

Prompt Sensitivity for Prompt Set 2: 0.6173056702313867
Prompt Sensitivity for Prompt Set 2: 0.3238119387223075
Prompt Sensitivity for Prompt Set 2: 0.24405630541344484
Prompt Sensitivity Index (POSIX): 0.39505797145571303
Prompt Sensitivity Index: 0.3951

--- Prompt Group 1 ---
Prompt: Q: What is the capital of France?
A: 
Response:  The capital of France is Paris.  
Logprobs: [-7.3389575749170035, -10.418194237689022, -18.44031661748886]
Prompt: Q: WHat is te capital city of France?
A: 
Response:  I don't know. I don't know
Logprobs: [-12.146543082839344, -10.77430819813162, -18.70745277404785]
Prompt: Q: what is teh cpital of france??
A: 
Response:  I think it's a bit of a mis
Logprobs: [-26.656580751470756, -13.104215703555383, -18.079093247652054]
Sensitivity: 0.6173056702313867

--- Prompt Group 2 ---
Prompt: Q: What is the national animal of India?
A: 
Response:  India is a country of animals.  
Logprobs: [-13.596061000833288, -18.71247360110283, -15.023666113615036]
Prompt: Q: 

In [10]:
# Load uploaded JSON (use your filename!)
with open("/content/alpaca_prompts.json", "r") as f:
    raw_data = json.load(f)

In [11]:
print("GPT2")
# Extract paraphrased prompts (testing first 2 groups now)
intent_aligned_prompts = [entry["prompts"] for entry in raw_data[:2]]

# Load model
# EleutherAI/gpt-neo-125M, mistralai/Mistral-7B-Instruct-v0.2
model = HFModel("openai-community/gpt2")

# Config
config = PosixConfig(max_new_tokens=20, batched=True)

# Run
score, trace = get_prompt_sensitivity(model, intent_aligned_prompts, config)
print(f"\nPSI score: {score:.4f}")

=====
GPT2
Setting pad token to <|endoftext|>


  0%|          | 0/2 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.



PSI score: 0.4396


In [12]:
print("Total prompts in trace:", len(trace.prompts))
print("Total responses in trace:", len(trace.responses))

# Check first 3 prompts and responses
for i in range(min(3, len(trace.prompts))):
    print(f"\nPrompt {i}: {trace.prompts[i]}")
    print(f"Response {i}: {trace.responses[i]}")

Total prompts in trace: 2
Total responses in trace: 2

Prompt 0: ['Q:How much do you know about Buddhism?\nA: ', 'Q:How much so yeou know about Buddhism ?\nA: ', 'Q:Hlow much do you know about Buddhism ?\nA: ', 'Q:How much do you know about huddhism ?\nA: ', 'Q:Hw muh dpo ypu knw abouht Buddhiism ?\nA: ', 'Q:yow much dmo oyu know about Buddhism ?\nA: ', 'Q:How much ddo you knod about Buddhism ?\nA: ', 'question: How much do you know about Buddhism?\nanswer: ', 'Question: How much do you know about Buddhism?\nAnswer: ', 'Q::: How much do you know about Buddhism?\nA::: ', 'q:::How much do you know about Buddhism?\na::: ', 'QUESTION:How much do you know about Buddhism?\nA: ', 'Q: How much do you know about Buddhism?\nAnswer: ', 'q:How much do you know about Buddhism?\na: ', 'Q: What is your level of experience with Buddhism?\nA: ', 'Q: How much information do you have about Buddhism?\nA: ', 'Q: What is your awareness of Buddhism?\nA: ', 'Q: What is your level of familiarity with Buddhism?

In [14]:
# Extract paraphrased prompts (first 2 groups for now)
intent_aligned_prompts = [entry["prompts"] for entry in raw_data[:2]]

# Load model
# EleutherAI/gpt-neo-125M, mistralai/Mistral-7B-Instruct-v0.2
model = HFModel("EleutherAI/gpt-neo-125M")

# Run
score, trace = get_prompt_sensitivity(model, intent_aligned_prompts, config)
print(f"\nPSI score: {score:.4f}")

print("Total prompts in trace:", len(trace.prompts))
print("Total responses in trace:", len(trace.responses))

# Check first 3 prompts and responses
for i in range(min(3, len(trace.prompts))):
    print(f"\nPrompt {i}: {trace.prompts[i]}")
    print(f"→ Response {i}: {trace.responses[i]}")

Setting pad token to <|endoftext|>


  0%|          | 0/2 [00:00<?, ?it/s]


PSI score: 0.8236
Total prompts in trace: 2
Total responses in trace: 2

Prompt 0: ['Q:How much do you know about Buddhism?\nA: ', 'Q:How much so yeou know about Buddhism ?\nA: ', 'Q:Hlow much do you know about Buddhism ?\nA: ', 'Q:How much do you know about huddhism ?\nA: ', 'Q:Hw muh dpo ypu knw abouht Buddhiism ?\nA: ', 'Q:yow much dmo oyu know about Buddhism ?\nA: ', 'Q:How much ddo you knod about Buddhism ?\nA: ', 'question: How much do you know about Buddhism?\nanswer: ', 'Question: How much do you know about Buddhism?\nAnswer: ', 'Q::: How much do you know about Buddhism?\nA::: ', 'q:::How much do you know about Buddhism?\na::: ', 'QUESTION:How much do you know about Buddhism?\nA: ', 'Q: How much do you know about Buddhism?\nAnswer: ', 'q:How much do you know about Buddhism?\na: ', 'Q: What is your level of experience with Buddhism?\nA: ', 'Q: How much information do you have about Buddhism?\nA: ', 'Q: What is your awareness of Buddhism?\nA: ', 'Q: What is your level of familia

In [15]:
print("Mistralai")

# Extract paraphrased prompts (first 2 groups for now)
intent_aligned_prompts = [entry["prompts"] for entry in raw_data[:2]]

# Load model
# EleutherAI/gpt-neo-125M, mistralai/Mistral-7B-Instruct-v0.2
model = HFModel("mistralai/Mistral-7B-Instruct-v0.2")

# Run
score, trace = get_prompt_sensitivity(model, intent_aligned_prompts, config)
print(f"\nPSI score: {score:.4f}")

print("Total prompts in trace:", len(trace.prompts))
print("Total responses in trace:", len(trace.responses))

# Check first 3 prompts and responses
for i in range(min(3, len(trace.prompts))):
    print(f"\nPrompt {i}: {trace.prompts[i]}")
    print(f"→ Response {i}: {trace.responses[i]}")

Mistralai


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting pad token to <unk>


  0%|          | 0/2 [00:00<?, ?it/s]


PSI score: 0.2828
Total prompts in trace: 2
Total responses in trace: 2

Prompt 0: ['Q:How much do you know about Buddhism?\nA: ', 'Q:How much so yeou know about Buddhism ?\nA: ', 'Q:Hlow much do you know about Buddhism ?\nA: ', 'Q:How much do you know about huddhism ?\nA: ', 'Q:Hw muh dpo ypu knw abouht Buddhiism ?\nA: ', 'Q:yow much dmo oyu know about Buddhism ?\nA: ', 'Q:How much ddo you knod about Buddhism ?\nA: ', 'question: How much do you know about Buddhism?\nanswer: ', 'Question: How much do you know about Buddhism?\nAnswer: ', 'Q::: How much do you know about Buddhism?\nA::: ', 'q:::How much do you know about Buddhism?\na::: ', 'QUESTION:How much do you know about Buddhism?\nA: ', 'Q: How much do you know about Buddhism?\nAnswer: ', 'q:How much do you know about Buddhism?\na: ', 'Q: What is your level of experience with Buddhism?\nA: ', 'Q: How much information do you have about Buddhism?\nA: ', 'Q: What is your awareness of Buddhism?\nA: ', 'Q: What is your level of familia